In [9]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor


train_data = pd.read_csv('rosneft/train_data.csv')
gtm = pd.read_csv('rosneft/gtm.csv')
coords = pd.read_csv('rosneft/coords.csv')
sampl_sab = pd.read_csv('rosneft/sample_submission.csv')
train_data["MEASURED_IN_DATE"] = pd.to_datetime(train_data["MEASURED_IN_DATE"])


# all_data = pd.read_csv('rosneft/gotov_ebat.csv', index_col='Unnamed: 0')
all_data = pd.read_csv('rosneft/gotov_ebat.csv', index_col='Unnamed: 0')
all_data["MEASURED_IN_DATE"] = pd.to_datetime(all_data["MEASURED_IN_DATE"])
all_data_w_d = all_data.drop(columns=['MEASURED_IN_DATE'])

In [10]:
all_data

,MEASURED_IN_DATE,CHARWORK,LIQ_RATE,WATER_CUT,P_ZAB,INTAKE,WELL_NAME,time_id
17437,2016-08-02,0.0,690.0,0.5,159.19,0.0,aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...,0
17438,2016-08-03,0.0,690.0,0.5,159.19,0.0,aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...,1
17439,2016-08-04,0.0,690.0,0.5,153.19,0.0,aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...,2
17440,2016-08-05,0.0,780.0,0.5,151.21,0.0,aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...,3
17441,2016-08-06,0.0,780.0,0.5,151.21,0.0,aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...,4
...,...,...,...,...,...,...,...,...
104377,2022-05-31,0.0,339.0,70.0,55.69,0.0,18ea8ca26bcabaac4fd073a00ba741a627f3649a6983a7...,2128
53662,2022-05-31,0.0,225.0,58.0,75.48,0.0,b04d54ff8a670262e81f0e0b514a32e03baa85200cd9f2...,2128
54379,2022-05-31,0.0,170.0,91.0,178.15,0.0,b694e479af091b82198c24a43195efbb9578a4f0e2e163...,2128
104892,2022-05-31,0.0,208.0,88.0,113.02,0.0,139e20fe864acf1a4b831ac25d417fe70725cbee85197c...,2128


In [11]:
def predict_step_by_step(model, row, days):
    forecasting_for_days = []
    new_arr_to_predict = row.to_numpy().tolist()
    for i in range(days):
        tensor_to_predict = np.array(new_arr_to_predict)

        predict = model.predict(tensor_to_predict)[0]
        forecasting_for_days.append(predict)
        new_arr_to_predict = [new_arr_to_predict[0][:-1]]
        new_arr_to_predict[0].insert(0, forecasting_for_days[-1])
    
    return forecasting_for_days

def lag_features_liquid_rate(df, lags):
    for lag in lags:
        df[f"lag_t-{lag}"] = df.groupby(["CHARWORK"])["LIQ_RATE"].transform(
            lambda x: x.shift(lag))
    return df

def lag_features_water_cut(df, lags):
    for lag in lags:
        df[f"lag_t-{lag}"] = df.groupby(["CHARWORK"])["WATER_CUT"].transform(
            lambda x: x.shift(lag))
    return df

In [35]:
def get_rf_cb_predictions(DATA_DIR, LAG_WINDOW_SIZE = 90) -> pd.DataFrame:

    all_data = pd.read_csv(DATA_DIR + '/gotov_ebat.csv')
    all_data["MEASURED_IN_DATE"] = pd.to_datetime(all_data["MEASURED_IN_DATE"])
    to_drop = ['P_ZAB', 'INTAKE', 'Unnamed: 0', 'time_id']
    all_data = all_data.drop(columns=to_drop)


    rf_to_do_dict = {
        'debcf7a160692239563af8a9c8ba32c4ba067e747f19c8073055e2be5aeb4022': 253,
         '114183900f2d540609911971f3ae3f3d42ab8b4fa9252d0c4076b00d482c1594': 216,
         '8d4b4b4f0140179b1c94722f3d39100bcde8f3c39415983813c74bd52cf1ab3c': 336,
         '2ceb0e2b3c28cc1b3c3f8dec7ad56148dbf6ea77f9ffc832f4c4b9ce845d9bb7': 133
    }

    rf_date_miss_dict = {
        key: all_data[all_data.WELL_NAME == key] \
        for key in rf_to_do_dict.keys()
    }
    
    
    rf_forecasts_dict_WC = dict()
    best_wc_models = []

    for c, i in enumerate(rf_date_miss_dict):
        train_data = rf_date_miss_dict[i].drop(columns=['WELL_NAME', 'LIQ_RATE'])
        train_with_lags = lag_features_water_cut(train_data, lags=[*range(1, LAG_WINDOW_SIZE + 1)])
        train_with_lags = train_with_lags.dropna()
        y = train_with_lags['WATER_CUT']
        if len(train_with_lags['MEASURED_IN_DATE']) == 0:
            continue
        else:
            last_date = np.array(train_with_lags['MEASURED_IN_DATE'])[-1]
            train_with_lags = train_with_lags.drop(columns=['WATER_CUT', 'CHARWORK'
                                                            , 'MEASURED_IN_DATE'])
            index_to_predict = pd.date_range(start=last_date + np.timedelta64(1, 'D')
                                             , end=last_date + np.timedelta64(rf_to_do_dict[i], 'D'))
            model = RandomForestRegressor(n_estimators=110, min_samples_split=5
                                          , min_samples_leaf=3, random_state=1984)        
            model.fit(train_with_lags, y)
            forecast = pd.Series(predict_step_by_step(model, train_with_lags.tail(1)
                                                      , rf_to_do_dict[i])
                                 , index=index_to_predict)
            rf_forecasts_dict_WC[i] = forecast
#             print('Score:\t', mean_squared_error(model.predict(train_with_lags), y))
    
    
    def drop_constant_columns(df):
        return df.loc[:, (df != df.iloc[0]).any()]


    cb_forecasts_dict_LR = dict()
    # best_wc_cbrs = []
    scores = []


    rf_forecasts_dict_LR = dict()
    # best_lr_models = []

    for c, i in enumerate(rf_date_miss_dict):
        train_data = rf_date_miss_dict[i].drop(columns=['WELL_NAME', 'WATER_CUT'])
        train_with_lags = lag_features_liquid_rate(train_data
                                                   , lags=[*range(1, LAG_WINDOW_SIZE + 1)])
        train_with_lags = train_with_lags.dropna()
        y = train_with_lags['LIQ_RATE']
        if len(train_with_lags['MEASURED_IN_DATE']) == 0:
            continue
        else:
            last_date = np.array(train_with_lags['MEASURED_IN_DATE'])[-1]
            train_with_lags = train_with_lags.drop(columns=['LIQ_RATE'
                                                            , 'MEASURED_IN_DATE', 'CHARWORK'])
            index_to_predict = pd.date_range(start=last_date + np.timedelta64(1, 'D')
                                             , end=last_date \
                                             + np.timedelta64(rf_to_do_dict[i], 'D'))

            model = CatBoostRegressor(iterations=1400, learning_rate=7e-3
                                              , max_depth=10, random_state=1984)
            model.fit(train_with_lags, y, metric_period=200)
            forecast = pd.Series(predict_step_by_step(model, train_with_lags.tail(1)
                                                      , rf_to_do_dict[i])
                                 , index=index_to_predict)
            cb_forecasts_dict_LR[i] = forecast
            scores.append(mean_squared_error(model.predict(train_with_lags), y))
    #         best__cbrs.append(search.best_estimator_)
    
    df = pd.DataFrame()
    check = set()
    for i in rf_forecasts_dict_WC:
        frcst_for_well_l_r = pd.DataFrame()
        frcst_for_well_w_c = pd.DataFrame()

        frcst_for_well_w_c['MEASURED_IN_DATE'] = rf_forecasts_dict_WC[i].index
        frcst_for_well_l_r['MEASURED_IN_DATE'] = cb_forecasts_dict_LR[i].index
        frcst_for_well_w_c['WELL_NAME'] = i
        frcst_for_well_l_r['WELL_NAME'] = i
        frcst_for_well_w_c = frcst_for_well_w_c.set_index('MEASURED_IN_DATE')
        frcst_for_well_l_r = frcst_for_well_l_r.set_index('MEASURED_IN_DATE')
        frcst_for_well_l_r['LIQ_RATE'] = cb_forecasts_dict_LR[i]
        frcst_for_well_w_c['WATER_CUT'] = rf_forecasts_dict_WC[i]
        frcst_for_well = frcst_for_well_l_r.merge(frcst_for_well_w_c
                                                  , on=['MEASURED_IN_DATE', 'WELL_NAME']
                                                  , how='inner')
        if i not in check:
            check.add(i)
        else:
            print(i)
        df = pd.concat([df, frcst_for_well])
    df = df.reset_index()
    
    return df

In [36]:
def get_rf_cb_predictions(DATA_DIR, LAG_WINDOW_SIZE = 90) -> pd.DataFrame:

    all_data = pd.read_csv(DATA_DIR + '/gotov_ebat.csv')
    all_data["MEASURED_IN_DATE"] = pd.to_datetime(all_data["MEASURED_IN_DATE"])
    to_drop = ['P_ZAB', 'INTAKE', 'Unnamed: 0', 'time_id']
    all_data = all_data.drop(columns=to_drop)

    # I HATE NIG-
    rf_to_do_dict = {
        'debcf7a160692239563af8a9c8ba32c4ba067e747f19c8073055e2be5aeb4022': 253,
         '114183900f2d540609911971f3ae3f3d42ab8b4fa9252d0c4076b00d482c1594': 216,
         '8d4b4b4f0140179b1c94722f3d39100bcde8f3c39415983813c74bd52cf1ab3c': 336,
         '2ceb0e2b3c28cc1b3c3f8dec7ad56148dbf6ea77f9ffc832f4c4b9ce845d9bb7': 133
    }

    rf_date_miss_dict = {
        key: all_data[all_data.WELL_NAME == key] \
        for key in rf_to_do_dict.keys()
    }
    rf_forecasts_dict_WC = dict()
    best_wc_models = []
    for c, i in enumerate(rf_date_miss_dict):
        train_data = rf_date_miss_dict[i].drop(columns=['WELL_NAME', 'LIQ_RATE'])
        train_with_lags = lag_features_water_cut(train_data, lags=[*range(1, LAG_WINDOW_SIZE + 1)])
        train_with_lags = train_with_lags.dropna()
        y = train_with_lags['WATER_CUT']
        if len(train_with_lags['MEASURED_IN_DATE']) == 0:
            continue
        else:
            last_date = np.array(train_with_lags['MEASURED_IN_DATE'])[-1]
            train_with_lags = train_with_lags.drop(columns=['WATER_CUT', 'CHARWORK'
                                                            , 'MEASURED_IN_DATE'])
            index_to_predict = pd.date_range(start=last_date + np.timedelta64(1, 'D')
                                             , end=last_date + np.timedelta64(rf_to_do_dict[i], 'D'))
            model = RandomForestRegressor(n_estimators=110, min_samples_split=5
                                          , min_samples_leaf=3, random_state=1984)        
            model.fit(train_with_lags, y)
            forecast = pd.Series(predict_step_by_step(model, train_with_lags.tail(1)
                                                      , rf_to_do_dict[i])
                                 , index=index_to_predict)
            rf_forecasts_dict_WC[i] = forecast
#             print('Score:\t', mean_squared_error(model.predict(train_with_lags), y))
    
    
    def drop_constant_columns(df):
        return df.loc[:, (df != df.iloc[0]).any()]


    cb_forecasts_dict_LR = dict()
    # best_wc_cbrs = []
    scores = []
    rf_forecasts_dict_LR = dict()
    # best_lr_models = []

    for c, i in enumerate(rf_date_miss_dict):
        train_data = rf_date_miss_dict[i].drop(columns=['WELL_NAME', 'WATER_CUT'])
        train_with_lags = lag_features_liquid_rate(train_data
                                                   , lags=[*range(1, LAG_WINDOW_SIZE + 1)])
        train_with_lags = train_with_lags.dropna()
        y = train_with_lags['LIQ_RATE']
        if len(train_with_lags['MEASURED_IN_DATE']) == 0:
            continue
        else:
            last_date = np.array(train_with_lags['MEASURED_IN_DATE'])[-1]
            train_with_lags = train_with_lags.drop(columns=['LIQ_RATE'
                                                            , 'MEASURED_IN_DATE', 'CHARWORK'])
            index_to_predict = pd.date_range(start=last_date + np.timedelta64(1, 'D')
                                             , end=last_date \
                                             + np.timedelta64(rf_to_do_dict[i], 'D'))

            model = CatBoostRegressor(iterations=1400, learning_rate=7e-3
                                              , max_depth=10, random_state=1984)
            model.fit(train_with_lags, y, metric_period=200)
            forecast = pd.Series(predict_step_by_step(model, train_with_lags.tail(1)
                                                      , rf_to_do_dict[i])
                                 , index=index_to_predict)
            cb_forecasts_dict_LR[i] = forecast
            scores.append(mean_squared_error(model.predict(train_with_lags), y))
    #         best__cbrs.append(search.best_estimator_)
    
    
    df = pd.DataFrame()
    check = set()
    for i in rf_forecasts_dict_WC:
        frcst_for_well_l_r = pd.DataFrame()
        frcst_for_well_w_c = pd.DataFrame()
        frcst_for_well_w_c['MEASURED_IN_DATE'] = rf_forecasts_dict_WC[i].index
        frcst_for_well_l_r['MEASURED_IN_DATE'] = cb_forecasts_dict_LR[i].index
        frcst_for_well_w_c['WELL_NAME'] = i
        frcst_for_well_l_r['WELL_NAME'] = i
        frcst_for_well_w_c = frcst_for_well_w_c.set_index('MEASURED_IN_DATE')
        frcst_for_well_l_r = frcst_for_well_l_r.set_index('MEASURED_IN_DATE')
        frcst_for_well_l_r['LIQ_RATE'] = cb_forecasts_dict_LR[i]
        frcst_for_well_w_c['WATER_CUT'] = rf_forecasts_dict_WC[i]
        frcst_for_well = frcst_for_well_l_r.merge(frcst_for_well_w_c
                                                  , on=['MEASURED_IN_DATE', 'WELL_NAME']
                                                  , how='inner')
        if i not in check:
            check.add(i)
        else:
            print(i)
        df = pd.concat([df, frcst_for_well])
    df = df.reset_index()
    
    return df

0:	learn: 40.7446887	total: 32.9ms	remaining: 46s
200:	learn: 13.4435697	total: 6.7s	remaining: 40s
400:	learn: 6.5421804	total: 13.6s	remaining: 34s
600:	learn: 4.8979563	total: 20.4s	remaining: 27.2s
800:	learn: 4.3135647	total: 27.9s	remaining: 20.9s
1000:	learn: 3.9644037	total: 35.9s	remaining: 14.3s
1200:	learn: 3.7153399	total: 43.5s	remaining: 7.21s
1399:	learn: 3.4747430	total: 51.4s	remaining: 0us
0:	learn: 23.6993047	total: 1.17ms	remaining: 1.64s
200:	learn: 12.5519917	total: 772ms	remaining: 4.61s
400:	learn: 7.7439892	total: 1.53s	remaining: 3.82s
600:	learn: 5.4028542	total: 2.38s	remaining: 3.16s
800:	learn: 4.0690680	total: 3.2s	remaining: 2.39s
1000:	learn: 3.1803532	total: 3.95s	remaining: 1.57s
1200:	learn: 2.5718623	total: 4.66s	remaining: 772ms
1399:	learn: 2.1601799	total: 5.38s	remaining: 0us
0:	learn: 38.6801650	total: 1.05ms	remaining: 1.47s
200:	learn: 16.5448036	total: 669ms	remaining: 3.99s
400:	learn: 7.5175734	total: 1.34s	remaining: 3.35s
600:	learn: 3.8

,MEASURED_IN_DATE,WELL_NAME,LIQ_RATE,WATER_CUT
0,2022-01-20,debcf7a160692239563af8a9c8ba32c4ba067e747f19c8...,435.535228,90.511002
1,2022-01-21,debcf7a160692239563af8a9c8ba32c4ba067e747f19c8...,434.564190,90.502407
2,2022-01-22,debcf7a160692239563af8a9c8ba32c4ba067e747f19c8...,434.619615,90.531591
3,2022-01-23,debcf7a160692239563af8a9c8ba32c4ba067e747f19c8...,433.021415,90.550742
4,2022-01-24,debcf7a160692239563af8a9c8ba32c4ba067e747f19c8...,431.159840,90.579024
...,...,...,...,...
933,2022-09-25,2ceb0e2b3c28cc1b3c3f8dec7ad56148dbf6ea77f9ffc8...,90.705456,7.545065
934,2022-09-26,2ceb0e2b3c28cc1b3c3f8dec7ad56148dbf6ea77f9ffc8...,90.687178,7.545065
935,2022-09-27,2ceb0e2b3c28cc1b3c3f8dec7ad56148dbf6ea77f9ffc8...,90.659510,7.545065
936,2022-09-28,2ceb0e2b3c28cc1b3c3f8dec7ad56148dbf6ea77f9ffc8...,90.721293,7.545065


In [16]:
all_data = pd.read_csv('rosneft/gotov_ebat.csv')
all_data["MEASURED_IN_DATE"] = pd.to_datetime(all_data["MEASURED_IN_DATE"])
to_drop = ['P_ZAB', 'INTAKE', 'Unnamed: 0', 'time_id']
all_data = all_data.drop(columns=to_drop)

rf_date_miss_dict = dict()
for i in all_data.WELL_NAME.unique():
    rf_date_miss_dict[i] = all_data[all_data['WELL_NAME'] == i]
# rf_date_miss_dict

In [61]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def vaildate_timeseries(data, y, model):
    X = data[:len(data)-20]
    y_true = y[len(data)-20:]
    forecast = pd.Series(predict_step_by_step(model, X, 20))
    
    return mean_absolute_error(forecast, y_true)

def rijilya_predict(LAG_WINDOW_SIZE=60, forecast_horizon=336) -> pd.DataFrame:    
    rf_forecasts_dict_WC = dict()
    rf_forecasts_dict_LR = dict()
    for c, i in enumerate(rf_date_miss_dict):
        print(i)
        train_data = rf_date_miss_dict[i].drop(columns=['WELL_NAME', 'LIQ_RATE'])
        train_with_lags = lag_features_water_cut(train_data, lags=[*range(1, LAG_WINDOW_SIZE + 1)])
        train_with_lags = train_with_lags.dropna()
        y = train_with_lags['WATER_CUT']
        if len(y) == 0:
            continue
        last_date = np.array(train_with_lags['MEASURED_IN_DATE'])[-1]
        index_to_predict = pd.date_range(start=last_date + np.timedelta64(1, 'D'), end=last_date + np.timedelta64(forecast_horizon, 'D'))
        train_with_lags = train_with_lags.drop(columns=['WATER_CUT', 'CHARWORK', 'MEASURED_IN_DATE'])
        model = RandomForestRegressor(n_estimators=110, min_samples_split=5, min_samples_leaf=3, random_state=2077)        
        model.fit(train_with_lags, y)
        forecast = pd.Series(predict_step_by_step(model, train_with_lags.tail(1), forecast_horizon), index=index_to_predict)
        rf_forecasts_dict_WC[i] = forecast
        print("validate on LR", vaildate_timeseries(train_with_lags, y, model))
        
        train_data = rf_date_miss_dict[i].drop(columns=['WELL_NAME', 'WATER_CUT'])
        train_with_lags = lag_features_liquid_rate(train_data, lags=[*range(1, LAG_WINDOW_SIZE + 1)])
        train_with_lags = train_with_lags.dropna()
        y = train_with_lags['LIQ_RATE']
        train_with_lags = train_with_lags.drop(columns=['LIQ_RATE', 'MEASURED_IN_DATE', 'CHARWORK'])
        model = RandomForestRegressor(n_estimators=110, min_samples_split=5, min_samples_leaf=3, random_state=2077)        
        model.fit(train_with_lags, y)
        print("validate on WC", vaildate_timeseries(train_with_lags, y, model))
        forecast = pd.Series(predict_step_by_step(model, train_with_lags.tail(1), forecast_horizon), index=index_to_predict)
        rf_forecasts_dict_LR[i] = forecast
        
        return rf_forecasts_dict_LR, rf_forecasts_dict_WC


In [63]:
rijilya_predict()

{'aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac29b1ac755b9a9b31cb8':        MEASURED_IN_DATE  CHARWORK  LIQ_RATE  WATER_CUT  \
 0            2016-08-02       0.0     690.0        0.5   
 1            2016-08-03       0.0     690.0        0.5   
 2            2016-08-04       0.0     690.0        0.5   
 3            2016-08-05       0.0     780.0        0.5   
 4            2016-08-06       0.0     780.0        0.5   
 ...                 ...       ...       ...        ...   
 121664       2022-05-27       0.0     758.0       92.0   
 121748       2022-05-28       0.0     758.0       92.0   
 122032       2022-05-29       0.0     758.0       92.0   
 122174       2022-05-30       0.0     758.0       92.0   
 122369       2022-05-31       0.0     758.0       92.0   
 
                                                 WELL_NAME  
 0       aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...  
 1       aebc60b8446bd0a77ff9f51006a8d675f29b1a90d5fac2...  
 2       aebc60b8446bd0a77ff9f51006a8d6